In [127]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 

In [128]:
sales_train=pd.read_csv("D:/ClassNotes_BA/Cloud/kaggle/sales_train.csv")

In [129]:
sales_train.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


In [130]:
sales_train.shape

(2935849, 6)

In [131]:
test= pd.read_csv("D:/ClassNotes_BA/Cloud/kaggle/test.csv")
test=test.drop_duplicates()
df_test=test

In [132]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [133]:
sample_sub=pd.read_csv("D:/ClassNotes_BA/Cloud/kaggle/sample_submission.csv")

In [134]:
sample_sub.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [135]:
items= pd.read_csv("D:/ClassNotes_BA/Cloud/kaggle/items.csv")

In [136]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [137]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [138]:
sales_train.isna().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [139]:
sales_train["date"].value_counts()

28.12.2013    9434
29.12.2013    9335
30.12.2014    9324
30.12.2013    9138
31.12.2014    8347
              ... 
14.05.2015    1174
28.04.2015    1154
06.10.2015    1152
13.05.2015    1128
12.05.2015    1123
Name: date, Length: 1034, dtype: int64

In [140]:
sales_train['date'] =  pd.to_datetime(sales_train['date'], infer_datetime_format=True)

In [141]:
df_train=sales_train

In [142]:
df_train["year"]=df_train['date'].dt.year
df_train["Month"]= df_train['date'].dt.month
df_train["day"]=df_train['date'].dt.day

In [143]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,Month,day
0,2013-02-01,0,59,22154,999.00,1.0,2013,2,1
1,2013-03-01,0,25,2552,899.00,1.0,2013,3,1
2,2013-05-01,0,25,2552,899.00,-1.0,2013,5,1
3,2013-06-01,0,25,2554,1709.05,1.0,2013,6,1
4,2013-01-15,0,25,2555,1099.00,1.0,2013,1,15


In [144]:
df_train.drop(["date","day"],axis=1,inplace=True)

In [145]:
df_train.groupby(["year","Month","shop_id",'item_id'])["item_cnt_day","item_price"].sum()

C:\Users\dell\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


item_cnt_day  item_price
year Month shop_id item_id                          
2013 1     0       32                4.0       663.0
                   33                2.0       694.0
                   35                2.0       494.0
                   43                1.0       221.0
                   51                2.0       257.0
...                                  ...         ...
2015 12    59      21421             1.0        19.0
                   21669             1.0       299.0
                   21872             1.0       148.0
                   21968            -1.0       949.0
                   22088             2.0       119.0

[1799395 rows x 2 columns]